<a href="https://colab.research.google.com/github/Haripriya-AIML/Priya/blob/main/AIML_A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Implement Logistic Regression Model Using Titanic Ship Dataset

Part 1 - Import the required Python, Pandas, Matplotlib, Seaborn packages.
[CO2]
1. Load the Titanic Ship Passengers dataset into a dataframe using pandas
2. Check the data types of each feature(column) in the dataset.
3. Generate a summary of the dataset for min, max, stddev,
quartile vales for 25%,50%,75%,90%,
4. List the names of columns/features in the dataset
5. Perform Exploratory analysis - plot numeric features, check relative size of survived/un-
survived, check if any pattern on gender. passenger class, class-wise survival rate, sibil-
ings, overall age distribution, class-wise age distribution - apply bar plot, histogram,
box plots to visualize.
6. Perform data wrangling - impute age data, drop unnecessary features, recode the cate-
gorical features to a class

In [8]:
#import nbconvert #recode the dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

train = pd.read_csv('/content/titanic_train.csv') # Training set is already available
train.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/titanic_train.csv'

Check basic info about the dat set including missing value

In [4]:
train.info(verbose=True)

NameError: name 'train' is not defined

In [ ]:
d=train.describe(percentiles=[.25,.5])
d

Exploratory analysis and plots
Plot a bar diagram to check the number of numeric entries From the bar diagram, it shows that there are some age entries missing

In [10]:
dT=d.T
dT.plot.bar(y='count')
plt.title("Bar plot of the count of numeric features",
          fontsize=17)
plt.xlabel("Numeric features",fontsize=17)
plt.ylabel("Count",fontsize=17)

NameError: name 'd' is not defined

check the relative size of survived and not-survived

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',hue='Fare',data=train,
              palette='rainbow')
#sns.pairplot(train)

Is there a pattern for the survivability based on sex?

It looks like more female survived than males!

In [11]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',hue='Pclass',data=train,palette='rainbow')

NameError: name 'train' is not defined

Following code extracts and plots the fraction of passenger count that survived, by each class

In [5]:
f_class_survived=train.groupby('Pclass')['Survived'].mean()
f_class_survived=pd.DataFrame(f_class_survived)
f_class_survived.shape

#f_class_survived.plot.bar(y='Survived')
#sns.countplot(x='Survived',hue='Pclass',data=f_class_survived,palette='rainbow')
#plt.title("Fraction of passengers survived by class",fontsize=17)

NameError: name 'train' is not defined

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',hue='SibSp',data=train,palette='rainbow')

How does the overall age distribution look like?

In [6]:
train['Age'].hist()
plt.xlabel("Age of the passengers",fontsize=18)
plt.ylabel("Count",fontsize=18)
plt.title("Agewise histogram of the passengers",fontsize=17)

NameError: name 'train' is not defined

In [ ]:
plt.figure(figsize=(12, 10))
plt.xlabel("Passenger Class",fontsize=18)
plt.ylabel("Age",fontsize=18)
sns.boxplot(x='Pclass',y='Age',data=train,palette='winter')

In [ ]:
f_class_Age=train.groupby('Pclass')['Age'].mean()
f_class_Age = pd.DataFrame(f_class_Age)

f_class_Age.plot.bar(y='Age')
plt.title("Average age of passengers by class",fontsize=17)
plt.ylabel("Age (years)",fontsize=17)
plt.xlabel("Passenger class",fontsize=17)

In [7]:
a=list(f_class_Age['Age'])

def impute_age(cols):
  Age = cols[0]
  Pclass = cols[1]

  if pd.isnull(Age):
    if Pclass == 1:
      return a[0]
    elif Pclass == 2:
      return a[1]
    else:
      return a[2]
  else:
    return Age

NameError: name 'f_class_Age' is not defined

Apply the above-defined function and plot the count of numeric features

In [ ]:
train['Age'] = train[['Age','Pclass']].apply(impute_age,axis=1)
d=train.describe()

dT=d.T
dT.plot.bar(y='count')
plt.title("Bar plot of numeric features",fontsize=17)

Drop the 'Cabin' feature and any other null value

In [ ]:
train.drop('Cabin',axis=1,inplace=True)
train.dropna(inplace=True)
train.head()

Drop other unnecessary features
like 'Cabin','PassengerId', 'Name', 'Ticket'

In [ ]:
train.drop(['PassengerId','Name','Ticket'],axis=1,
           inplace=True)
train.dropna(inplace=True)
train.head()

Convert categorial feature like 'Sex'
and 'Embarked' to dummy variables

Use pandas 'get_dummies()' function

In [ ]:
sex = pd.get_dummies(train['Sex'],drop_first=True)
embark = pd.get_dummies(train['Embarked'],drop_first=True)

Now drop the 'Sex' and 'Embarked' columns and concatenate the new dummy variables

In [ ]:
train.drop(['Sex','Embarked'],axis=1,inplace=True)
train = pd.concat([train,sex,embark],axis=1)
train.head()

Logistic Regression model fit and prediction
Let's start by splitting our data into a training set and test set (there is another test.csv file that you can play around with in case you want to use all this data for training).

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    train.drop('Survived',axis=1),train['Survived'],
    test_size=0.20,random_state=111)

F1-score as a function of regularization (penalty) parameter


In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
nsimu=201
penalty=[0]*nsimu
logmodel=[0]*nsimu
predictions =[0]*nsimu
class_report = [0]*nsimu
f1=[0]*nsimu

for i in range(1,nsimu):
        logmodel[i] =(LogisticRegression(C=i/1000,tol=1e-4,
                                         max_iter=int(1e6),
                                         n_jobs=4))
        logmodel[i].fit(X_train,y_train)
        predictions[i] = logmodel[i].predict(X_test)
        class_report[i] = classification_report(y_test,
                                                predictions[i])
        l=class_report[i].split()
        f1[i] = l[len(l)-2]
        penalty[i]=1000/i

plt.scatter(penalty[1:len(penalty)-2],f1[1:len(f1)-2])
plt.title("F1-score vs. regularization parameter",fontsize=20)
plt.xlabel("Penalty parameter",fontsize=17)
plt.ylabel("F1-score on test data",fontsize=17)
plt.show()

NameError: name 'X_train' is not defined

F1-score as a function of test set size (fraction)

In [ ]:
nsimu=101
class_report = [0]*nsimu
f1=[0]*nsimu
test_fraction =[0]*nsimu
for i in range(1,nsimu):
        X_train, X_test, y_train, y_test = train_test_split(train.drop('Survived',axis=1),
                                                    train['Survived'], test_size=0.1+(i-1)*0.007,
                                                    random_state=111)
        logmodel =(LogisticRegression(C=1,tol=1e-4, max_iter=1000,n_jobs=4))
        logmodel.fit(X_train,y_train)
        predictions = logmodel.predict(X_test)
        class_report[i] = classification_report(y_test,predictions)
        l=class_report[i].split()
        f1[i] = l[len(l)-2]
        test_fraction[i]=0.1+(i-1)*0.007

plt.plot(test_fraction[1:len(test_fraction)-2],f1[1:len(f1)-2])
plt.title("F1-score vs. test set size (fraction)",fontsize=20)
plt.xlabel("Test set size (fraction)",fontsize=17)
plt.ylabel("F1-score on test data",fontsize=17)
plt.show()

F1-score as a function of random seed of test/train split

In [ ]:
nsimu=101
class_report = [0]*nsimu
f1=[0]*nsimu
random_init =[0]*nsimu
for i in range(1,nsimu):
        X_train, X_test, y_train, y_test = train_test_split(train.drop('Survived',axis=1),
                                                    train['Survived'], test_size=0.3,
                                                    random_state=i+100)
        logmodel =(LogisticRegression(C=1,tol=1e-5, max_iter=1000,n_jobs=4))
        logmodel.fit(X_train,y_train)
        predictions = logmodel.predict(X_test)
        class_report[i] = classification_report(y_test,predictions)
        l=class_report[i].split()
        f1[i] = l[len(l)-2]
        random_init[i]=i+100

plt.plot(random_init[1:len(random_init)-2],f1[1:len(f1)-2])
plt.title("F1-score vs. random initialization seed",fontsize=20)
plt.xlabel("Random initialization seed",fontsize=17)
plt.ylabel("F1-score on test data",fontsize=17)
plt.show()